In [2]:
import numpy as np
import pandas as pd
from math import sqrt, pi

# Function under the integral
def f(x):
    return np.exp(-x**2 / 2)

# Simpson's rule
def simpson(f, a, b, n):
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    fx = f(x)
    return h / 3 * (fx[0] + 2 * np.sum(fx[2:n:2]) + 4 * np.sum(fx[1:n:2]) + fx[n])

# Compute N(t) with convergence log
def compute_N_with_log(t, tol=1e-12):
    a, b = (0, t) if t >= 0 else (t, 0)
    logs = []
    n = 2
    I_old = simpson(f, a, b, n)
    logs.append((n, 0.5 + np.sign(t) * I_old / sqrt(2 * pi)))

    while True:
        n *= 2
        I_new = simpson(f, a, b, n)
        N_val = 0.5 + np.sign(t) * I_new / sqrt(2 * pi)
        logs.append((n, N_val))
        if abs(I_new - I_old) < tol:
            break
        I_old = I_new

    return pd.DataFrame(logs, columns=["n", f"N({t})"])

# Combine results into one DataFrame
df_list = [compute_N_with_log(t) for t in [0.1, 0.5, 1.0]]
merged = df_list[0]
for df in df_list[1:]:
    merged = pd.merge(merged, df, on="n", how="outer")

# Sort by n and format values
merged = merged.sort_values("n").reset_index(drop=True)


In [4]:
print(merged.to_latex(index=False, float_format="%.13f", escape=False))

\begin{tabular}{rrrr}
\toprule
n & N(0.1) & N(0.5) & N(1.0) \\
\midrule
2 & 0.5398278414043 & 0.6914733395314 & 0.8415290519963 \\
4 & 0.5398278375347 & 0.6914631235012 & 0.8413554878566 \\
8 & 0.5398278372931 & 0.6914625023982 & 0.8413454061391 \\
16 & 0.5398278372780 & 0.6914624638402 & 0.8413447871501 \\
32 & 0.5398278372771 & 0.6914624614343 & 0.8413447486335 \\
64 & 0.5398278372770 & 0.6914624612840 & 0.8413447462288 \\
128 & NaN & 0.6914624612746 & 0.8413447460786 \\
256 & NaN & 0.6914624612741 & 0.8413447460692 \\
512 & NaN & 0.6914624612740 & 0.8413447460686 \\
1024 & NaN & NaN & 0.8413447460685 \\
\bottomrule
\end{tabular}

